
1. Install spark and run a worker on your machine

2. Using Spark, inspect the Bank Marketing dataset

 a.  Print 10 random rows from the dataset

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand

# Initialize Spark session
spark = SparkSession.builder.appName("BankMarketingAnalysis").getOrCreate()

# Load the dataset (update path as needed)
df = spark.read.csv("bank-marketing.csv", header=True, inferSchema=True)

# Print 10 random rows
df.orderBy(rand()).show(10, truncate=False)


+---+------------+--------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job         |marital |education|default|balance|housing|loan|contact |day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+------------+--------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+-------+
|52 |admin.      |married |secondary|no     |1113   |no     |yes |cellular|8  |jul  |83      |1       |-1   |0       |unknown |no     |
|26 |management  |single  |tertiary |no     |1094   |yes    |no  |cellular|8  |sep  |137     |1       |-1   |0       |unknown |yes    |
|27 |technician  |single  |secondary|yes    |0      |no     |yes |cellular|8  |jul  |732     |1       |-1   |0       |unknown |yes    |
|49 |admin.      |married |secondary|no     |167    |no     |yes |cellular|9  |jul  |51      |1       |-1   |0       |unknown |no     |
|33 |management  |single  |tertiary |no     |224

2.b. What are the relative proportions of no and yes for deposit feature? (inspect also other qualitative variables)

In [17]:
print('proportion of \'yes\' is ', df.filter(df["deposit"] == "yes").count() / df.count())


proportion of 'yes' is  0.4738398136534671


2.c. Get descriptive statistics for numerical variables
 - Use relevant visualizations to inspect variables and relations between them

Answer the following questions

- Who is the client with the biggest balance?
- What is the proportion of clients who's balance is more than twice the average?
- Do people with higher education have a better chance to deposit?
- What are the best predictors for deposit?

In [6]:
df.orderBy('age').show(10, truncate=False)

+---+-------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job    |marital|education|default|balance|housing|loan|contact |day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+-------+
|18 |student|single |unknown  |no     |108    |no     |no  |cellular|10 |aug  |167     |1       |-1   |0       |unknown |yes    |
|18 |student|single |primary  |no     |608    |no     |no  |cellular|12 |aug  |267     |1       |-1   |0       |unknown |yes    |
|18 |student|single |unknown  |no     |108    |no     |no  |cellular|8  |sep  |169     |1       |-1   |0       |unknown |yes    |
|18 |student|single |primary  |no     |608    |no     |no  |cellular|13 |nov  |210     |1       |93   |1       |success |yes    |
|18 |student|single |unknown  |no     |108    |no     |no  |cellular|9  |feb  |92      |1 